In [1]:
import requests
from bs4 import BeautifulSoup
import os
import gzip
import shutil
import cartopy.crs as ccrs
import cartopy.feature
import cartopy
from metpy.plots import USCOUNTIES
from datetime import datetime, timedelta
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from pprint import pprint
from pysteps import io, nowcasts, rcparams
from pysteps.motion.lucaskanade import dense_lucaskanade
from pysteps.postprocessing.ensemblestats import excprob
from pysteps.utils import conversion, dimension, transformation
from pysteps.visualization import plot_precip_field
import matplotlib.colors as colors
import re
import pandas as pd
# import scikit-image

# Set nowcast parameters
n_ens_members = 20 #20
n_leadtimes = 7
seed = 20 #24
precip_thr = -10

#north plains
domain_name = 'north_plains_'
domain_lat = [40, 49]
domain_lon = [-110, -95]

#co
domain_name = 'co_'
domain_lat = [36.7, 41.5]
domain_lon = [-110, -101]

# #neb
# domain_name = 'neb_'
# domain_lat = [39, 45]
# domain_lon = [-106, -95]

# #US
# domain_name = 'us_'
# domain_lat = [29, 48]
# domain_lon = [-124, -67]

# #Midwest
# domain_name = 'Midwest_'
# domain_lat = [42, 48]
# domain_lon = [-96, -83]

# #custom
# domain_name = 'custom_'
# domain_lat = [33, 40]
# domain_lon = [-103, -90]

# # NW
# domain_name = 'nw_'
# domain_lat = [41, 49]
# domain_lon = [-124, -110]

# num_prev_files 
# n_ens_members 
# n_leadtimes 
# seed 
# execution_time
# method

Pysteps configuration file found at: C:\Users\16126\pysteps\pystepsrc



In [2]:
from pysteps import datasets
# from pysteps import io, conversion, dimension, transformation
import pysteps

datasets.info()
from pprint import pprint
import pysteps
pprint(pysteps.rcparams.data_sources["mrms_os"])


Available datasets:

Case     Event date             Source                                       

fmi      2016-09-28 14:45 UTC   Finish Meteorological Institute              
fmi2     2017-05-09 10:45 UTC   Finish Meteorological Institute              
mch      2015-05-15 15:45 UTC   MeteoSwiss                                   
mch2     2016-07-11 20:45 UTC   MeteoSwiss                                   
mch3     2017-01-31 09:45 UTC   MeteoSwiss                                   
opera    2018-08-24 18:00 UTC   OPERA                                        
knmi     2010-08-26 00:00 UTC   Royal Netherlands Meteorological Institute   
bom      2018-06-16 10:00 UTC   Australian Bureau of Meteorology             
mrms     2019-06-10 00:00 UTC   NSSL's Multi-Radar/Multi-Sensor System       
mrms_os  2023-05-07 23:00 UTC   NSSL's Multi-Radar/Multi-Sensor System       
{'fn_ext': 'grib2',
 'fn_pattern': 'MRMS_PrecipRate_00.00_%Y%m%d-%H%M%S',
 'importer': 'mrms_grib',
 'importer_kwargs':

## Download and process real-time MRMS Data

In [3]:
url = 'https://mrms.ncep.noaa.gov/data/2D/PrecipRate/'

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Find all the links on the page
links = soup.find_all('a')

In [4]:
# Filter the links to get the .gz files and select every 3rd file
gz_files = [link['href'] for link in links if link['href'].endswith('.gz')]
gz_files.sort(reverse=True)
selected_files = gz_files[::4][:5]
print(selected_files)

['MRMS_PrecipRate_00.00_20230520-143000.grib2.gz', 'MRMS_PrecipRate_00.00_20230520-142200.grib2.gz', 'MRMS_PrecipRate_00.00_20230520-141400.grib2.gz', 'MRMS_PrecipRate_00.00_20230520-140600.grib2.gz', 'MRMS_PrecipRate_00.00_20230520-135800.grib2.gz']


In [5]:
# from datetime import datetime
# import os
# Get the current date
current_date = datetime.now().date()

# Extract the current year, month, and day
current_year = str(current_date.year)
current_month = f"{current_date.month:02d}"
current_day = f"{current_date.day:02d}"

# Download the selected files
download_folder = os.path.join(r'C:\Users\16126\pysteps_data\mrms_os', current_year, current_month, current_day)

# Check if the path exists
if not os.path.exists(download_folder):
    # Create the directory
    os.makedirs(download_folder)
    print("Directory created:", download_folder)
else:
    print("Directory already exists:", download_folder)

# Get a list of all files in the folder
files = os.listdir(download_folder)

# Remove each file in the folder
for file in files:
    file_path = os.path.join(download_folder, file)
    os.remove(file_path)
    
for file in selected_files:
    file_url = url + file
    response = requests.get(file_url)
    
    # Create the file path in the download folder
    file_path = os.path.join(download_folder, file)
    
    # Save the file to the specified folder location
    with open(file_path, 'wb') as f:
        f.write(response.content)
        print(f"Downloaded: {file}")

Directory already exists: C:\Users\16126\pysteps_data\mrms_os\2023\05\20
Downloaded: MRMS_PrecipRate_00.00_20230520-143000.grib2.gz
Downloaded: MRMS_PrecipRate_00.00_20230520-142200.grib2.gz
Downloaded: MRMS_PrecipRate_00.00_20230520-141400.grib2.gz
Downloaded: MRMS_PrecipRate_00.00_20230520-140600.grib2.gz
Downloaded: MRMS_PrecipRate_00.00_20230520-135800.grib2.gz


In [6]:
input_directory = download_folder
output_directory = download_folder

# Get a list of all files in the input directory
files = os.listdir(input_directory)

# Filter the list to include only the .gz files
gz_files = [file for file in files if file.endswith('.gz')]

# Iterate over the .gz files and convert them to .grib2
for gz_file in gz_files:
    gz_path = os.path.join(input_directory, gz_file)
    grib2_file = os.path.splitext(gz_file)[0]
    grib2_path = os.path.join(output_directory, grib2_file)
    
    with gzip.open(gz_path, 'rb') as f_in:
        with open(grib2_path, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    
    print(f"Converted: {gz_file} -> {grib2_file}")
    # Remove the .gz file
    os.remove(gz_path)
    print(f"Removed: {gz_file}")

Converted: MRMS_PrecipRate_00.00_20230520-135800.grib2.gz -> MRMS_PrecipRate_00.00_20230520-135800.grib2
Removed: MRMS_PrecipRate_00.00_20230520-135800.grib2.gz
Converted: MRMS_PrecipRate_00.00_20230520-140600.grib2.gz -> MRMS_PrecipRate_00.00_20230520-140600.grib2
Removed: MRMS_PrecipRate_00.00_20230520-140600.grib2.gz
Converted: MRMS_PrecipRate_00.00_20230520-141400.grib2.gz -> MRMS_PrecipRate_00.00_20230520-141400.grib2
Removed: MRMS_PrecipRate_00.00_20230520-141400.grib2.gz
Converted: MRMS_PrecipRate_00.00_20230520-142200.grib2.gz -> MRMS_PrecipRate_00.00_20230520-142200.grib2
Removed: MRMS_PrecipRate_00.00_20230520-142200.grib2.gz
Converted: MRMS_PrecipRate_00.00_20230520-143000.grib2.gz -> MRMS_PrecipRate_00.00_20230520-143000.grib2
Removed: MRMS_PrecipRate_00.00_20230520-143000.grib2.gz


In [7]:
# Get a list of all files in the input directory
files = os.listdir(input_directory)
files_sorted = sorted(files, reverse=True)
gif_date_start = re.search(r"(\d{8}-\d{6})", files_sorted[0]).group()
# gif_date_start = gif_date_start.split('_')
# gif_date_start = gif_date_start[-1].split('.')[0]
print(files_sorted[0],gif_date_start)

MRMS_PrecipRate_00.00_20230520-143000.grib2 20230520-143000


In [8]:
# Example filename
filename = files_sorted[0]

# Example filename
filename = files_sorted[0]

# Extract year, month, day, hour, and minute from the filename
year = filename[22:26]
month = filename[26:28]
day = filename[28:30]
hour = filename[31:33]
minute = filename[33:35]

# Format the extracted values
formatted_datetime = f"{year}{month}{day}{hour}{minute}"

# Print the formatted datetime
print("Formatted datetime:", formatted_datetime)

Formatted datetime: 202305201430


In [9]:
date = datetime.strptime(formatted_datetime, "%Y%m%d%H%M")

# Subtract 8 minutes from the datetime object
new_date = date - timedelta(minutes=8)

# Format the new date as a string
new_date_string = new_date.strftime("%Y%m%d%H%M")

# Print the new date string
print("New date:", new_date_string)

New date: 202305201422


## Ingest and Map MRMS

In [10]:
# date = datetime.strptime(formatted_datetime, "%Y%m%d%H%M")
date = datetime.strptime("202305181432", "%Y%m%d%H%M")

data_source = "mrms_os"      
from pysteps.visualization.basemaps import plot_map_cartopy

# Load data source config
# data_source = "mrms_os"
root_path = rcparams.data_sources[data_source]["root_path"]
path_fmt = rcparams.data_sources[data_source]["path_fmt"]
fn_pattern = rcparams.data_sources[data_source]["fn_pattern"]
fn_ext = rcparams.data_sources[data_source]["fn_ext"]
importer_name = rcparams.data_sources[data_source]["importer"]
importer_kwargs = rcparams.data_sources[data_source]["importer_kwargs"]
timestep = rcparams.data_sources[data_source]["timestep"]
num_prev_files = 4

# Find the radar files in the archive
fns = io.find_by_date(
    date, root_path, path_fmt, fn_pattern, fn_ext, timestep, num_prev_files
)

# Read the data from the archive
importer = io.get_method(importer_name, "importer")
R, _, metadata = io.read_timeseries(fns, importer, **importer_kwargs)

# Convert to rain rate
# R, metadata = conversion.to_rainrate(R, metadata)

# Upscale data to 2 km to limit memory usage
# R, metadata = dimension.aggregate_fields_space(R, metadata, 2000)
    
for i in range(R.shape[0]):
    if i == 0:
        f_step = 32
    elif i == 1:
        f_step = 24
    elif i == 2:
        f_step = 16
    elif i == 3:
        f_step = 8
    elif i == 4:
        f_step = 0

    # Subtract 8 minutes from the datetime object
    new_date = date - timedelta(minutes=(num_prev_files - i) * 8)
    # Format the new date as a string
    new_date_string = new_date.strftime("%Y%m%d%H%M")
    
    # Get the extent values from metadata
    x1 = metadata['x1']
    x2 = metadata['x2']
    y2 = metadata['y1']
    y1 = metadata['y2']

    # Create a grid of x and y coordinates
    x = np.linspace(x1, x2, R.shape[2])
    y = np.linspace(y1, y2, R.shape[1])
    X, Y = np.meshgrid(x, y)

    bounds = [0.08, 0.16, 0.25,  0.40, 0.63, 1, 1.6, 2.5, 4, 6.3, 10, 16, 25, 40, 63, 100, 160, 220]
    norm = colors.BoundaryNorm(boundaries=bounds, ncolors=len(bounds))
    colormap = colors.ListedColormap(['#9e7c94', #grey
                                      '#650064','#ae00b0','#dc01da', #purples 5-15
                                      '#3531c3','#0b60f9', #blues
                                      '#009697', '#00c930','#65ff01','#98ff00','#c6fe00', #greens
                                      '#fffd04', #yellow
                                      '#ffc802', '#ffa000','#ff7d01', #orange
                                      '#df1a01','#8d0000' ]) #red           #65-75

    # Create the figure and subplots
    # fig, axs = plt.subplots(1, 2, figsize=(18, 9), dpi=200, subplot_kw={'projection': ccrs.PlateCarree()})
    fig, axs = plt.subplots(2, 1, figsize=(12, 12), dpi=200, subplot_kw={'projection': ccrs.PlateCarree()})

    # Add features to both subplots
    for ax in axs:
        ax.add_feature(USCOUNTIES.with_scale('5m'), linewidth=.5, edgecolor='darkgrey')
        ax.add_feature(cartopy.feature.STATES, linewidth=1, edgecolor='black')
        
        # Add features to both subplots
        # Set up the first subplot
        axs[0].set_title(new_date_string + " MRMS (-%i min)" % (f_step))
        axs[0].set_xlim(domain_lon)
        axs[0].set_ylim(domain_lat)

        # Set up the second subplot
        axs[1].set_title(new_date_string + " MRMS (-%i min)" % (f_step))
        axs[1].set_xlim(domain_lon)
        axs[1].set_ylim(domain_lat)

        # Plot in the first subplot
        axs[0].contourf(X, Y, R[i, :, :], levels=bounds, norm=norm, cmap=colormap)

        # Plot in the second subplot
        axs[1].contourf(X, Y, R[i, :, :], levels=bounds, norm=norm, cmap=colormap)
        
        # Plot 2.5mm contour
        axs[0].contour(X, Y, R[i, :, :], levels=[.254], colors='black')
        axs[1].contour(X, Y, R[i, :, :], levels=[.254], colors='black')

    # Save the figure
    file_name = str(i) + "_MRMS (-%i min)" % (f_step)
    mrms_radar_path = os.path.join(r'G:\My Drive\PowPow\Resort Graphs\forecast_radar\mrms', file_name)
    plt.savefig(mrms_radar_path + '.png', dpi=200, bbox_inches='tight')
    plt.close()
    
    
# Log-transform the data to unit of dBR, set the threshold to 0.1 mm/h,
# set the fill value to -15 dBR
# R, metadata = transformation.dB_transform(R, metadata, threshold=0.1, zerovalue=-15.0)

# Set missing values with the fill value
R[~np.isfinite(R)] = -15.0

## Create S-PROG

In [ ]:
import time
start_time = time.time()

# Estimate the motion field
from pysteps import motion
V = dense_lucaskanade(R)

nowcast_method = nowcasts.get_method("sprog")
R_f = nowcast_method(
    R[-3:, :, :],
    V,
    n_leadtimes,
    n_cascade_levels=6,
    precip_thr=precip_thr,
    extrap_method='semilagrangian',
)

end_time = time.time()
execution_time = end_time - start_time
method = 'STEPS S-PROG'
df = pd.read_csv(r"G:\My Drive\PowPow\Resort Graphs\forecast_radar\calc_duration.csv")
# Create a new row with the variables as columns
new_row = {"method": method,"num_prev_files": num_prev_files,"n_ens_members": n_ens_members,"n_leadtimes": n_leadtimes,"seed": seed,"execution_time": execution_time,}
# Convert the new row to a DataFrame
new_row_df = pd.DataFrame([new_row])
# Concatenate the original DataFrame with the new row DataFrame
df = pd.concat([df, new_row_df], ignore_index=True)
# Save the modified DataFrame to a new CSV file
df.to_csv(r"G:\My Drive\PowPow\Resort Graphs\forecast_radar\calc_duration.csv", index=False)

# Back-transform to rain rate
# R_f = transformation.dB_transform(R_f, threshold=-10.0, inverse=True)[0]

Computing S-PROG nowcast
------------------------

Inputs
------
input dimensions: 875x1750

Methods
-------
extrapolation:          semilagrangian
bandpass filter:        gaussian
decomposition:          fft
conditional statistics: no
probability matching:   cdf
FFT method:             numpy
domain:                 spatial

Parameters
----------
number of time steps:     7
parallel threads:         1
number of cascade levels: 6
order of the AR(p) model: 2
precip. intensity threshold: -10
************************************************
* Correlation coefficients for cascade levels: *
************************************************
-----------------------------------------
| Level |     Lag-1     |     Lag-2     |
-----------------------------------------
| 1     | 0.999996      | 0.998849      |
-----------------------------------------
| 2     | 0.999951      | 0.972260      |
-----------------------------------------
| 3     | 0.999522      | 0.962960      |
-----------------------

## Create S-PROG v2

In [ ]:
import time
start_time = time.time()

# Estimate the motion field
from pysteps import motion
V = dense_lucaskanade(R)

nowcast_method = nowcasts.get_method("sprog")
R_f_v2 = nowcast_method(
    R[-3:, :, :],
    V,
    n_leadtimes,
    n_cascade_levels=6,
    precip_thr=precip_thr,
    extrap_method='semilagrangian',
    probmatching_method="cdf"
)

end_time = time.time()
execution_time = end_time - start_time
method = 'STEPS S-PROG v2'
df = pd.read_csv(r"G:\My Drive\PowPow\Resort Graphs\forecast_radar\calc_duration.csv")
# Create a new row with the variables as columns
new_row = {"method": method,"num_prev_files": num_prev_files,"n_ens_members": n_ens_members,"n_leadtimes": n_leadtimes,"seed": seed,"execution_time": execution_time,}
# Convert the new row to a DataFrame
new_row_df = pd.DataFrame([new_row])
# Concatenate the original DataFrame with the new row DataFrame
df = pd.concat([df, new_row_df], ignore_index=True)
# Save the modified DataFrame to a new CSV file
df.to_csv(r"G:\My Drive\PowPow\Resort Graphs\forecast_radar\calc_duration.csv", index=False)

# Back-transform to rain rate
# R_f_v2 = transformation.dB_transform(R_f_v2, threshold=-10.0, inverse=True)[0]

## Plot S-PROG

In [ ]:
for i in range(R_f.shape[0]):
    f_step = i + 1
    
    # Add 8 minutes from the datetime object
    new_date = date + timedelta(minutes=(f_step)*8)
    # Format the new date as a string
    new_date_string = new_date.strftime("%Y%m%d%H%M")

    # Get the extent values from metadata
    x1 = metadata['x1']
    x2 = metadata['x2']
    y2 = metadata['y1']
    y1 = metadata['y2']

    # Create a grid of x and y coordinates
    x = np.linspace(x1, x2, R.shape[2])
    y = np.linspace(y1, y2, R.shape[1])
    X, Y = np.meshgrid(x, y)

    bounds = [0.08, 0.16, 0.25,  0.40, 0.63, 1, 1.6, 2.5, 4, 6.3, 10, 16, 25, 40, 63, 100, 160, 220]
    norm = colors.BoundaryNorm(boundaries=bounds, ncolors=len(bounds))
    colormap = colors.ListedColormap(['#9e7c94', #grey
                                      '#650064','#ae00b0','#dc01da', #purples 5-15
                                      '#3531c3','#0b60f9', #blues
                                      '#009697', '#00c930','#65ff01','#98ff00','#c6fe00', #greens
                                      '#fffd04', #yellow
                                      '#ffc802', '#ffa000','#ff7d01', #orange
                                      '#df1a01','#8d0000' ]) #red           #65-75
    
    # Create the figure and subplots
    # fig, axs = plt.subplots(1, 2, figsize=(18, 9), dpi=200, subplot_kw={'projection': ccrs.PlateCarree()})
    fig, axs = plt.subplots(2, 1, figsize=(12, 12), dpi=200, subplot_kw={'projection': ccrs.PlateCarree()})

    # Add features to both subplots
    for ax in axs:
        ax.add_feature(USCOUNTIES.with_scale('5m'), linewidth=.5, edgecolor='darkgrey')
        ax.add_feature(cartopy.feature.STATES, linewidth=1, edgecolor='black')

        # Set up the first subplot
        axs[0].set_title(new_date_string + " S-PROG (+ %i min)" % (f_step * timestep))
        axs[0].set_xlim(domain_lon)
        axs[0].set_ylim(domain_lat)

        # Set up the second subplot
        axs[1].set_title(new_date_string + " S-PROG v2 (+ %i min)" % (f_step * timestep)) #'Placeholder for LINDA Deterministic')
        axs[1].set_xlim(domain_lon)
        axs[1].set_ylim(domain_lat)

        # Plot in the subplots
        axs[0].contourf(X, Y, R_f[i, :, :], levels=bounds, norm=norm, cmap=colormap)
        axs[1].contourf(X, Y, R_f_v2[i, :, :], levels=bounds, norm=norm, cmap=colormap)
        
        # Plot 2.5mm contour
        axs[0].contour(X, Y, R_f[i, :, :], levels=[.254], colors='black')
        axs[1].contour(X, Y, R_f[i, :, :], levels=[.254], colors='black')

    file_name="S-PROG + S-PROG v2 (+ %i min)" % (f_step * timestep)
    mrms_radar_path = os.path.join(r'G:\My Drive\PowPow\Resort Graphs\forecast_radar\deterministic', file_name)
    plt.savefig(mrms_radar_path+'.png',dpi=200,bbox_inches='tight')
    plt.close()

## STEPS 

In [ ]:
import time
start_time = time.time()

# Estimate the motion field
V = dense_lucaskanade(R)

# The STEPS nowcast
nowcast_method = nowcasts.get_method("steps")
R_f_ens = nowcast_method(
    R[-3:, :, :],
    V,
    n_leadtimes,
    n_ens_members,
    n_cascade_levels=6,
    precip_thr=precip_thr,
    kmperpixel=2,
    timestep=timestep,
    # noise_method="nonparametric",
    vel_pert_method="bps",
    mask_method="incremental",
    seed=seed,
    extrap_method='semilagrangian',
    num_workers=2
)

# Back-transform to rain rates
# R_f_ens = transformation.dB_transform(R_f_ens, threshold=-10.0, inverse=True)[0]

end_time = time.time()
execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")
method = 'STEPS Ens'
df = pd.read_csv(r"G:\My Drive\PowPow\Resort Graphs\forecast_radar\calc_duration.csv")
# Create a new row with the variables as columns
new_row = {"method": method,"num_prev_files": num_prev_files,"n_ens_members": n_ens_members,"n_leadtimes": n_leadtimes,"seed": seed,"execution_time": execution_time,}

# Convert the new row to a DataFrame
new_row_df = pd.DataFrame([new_row])

# Concatenate the original DataFrame with the new row DataFrame
df = pd.concat([df, new_row_df], ignore_index=True)

# Save the modified DataFrame to a new CSV file
df.to_csv(r"G:\My Drive\PowPow\Resort Graphs\forecast_radar\calc_duration.csv", index=False)

## STEPS v2

In [ ]:
import time
start_time = time.time()

# Estimate the motion field
V = dense_lucaskanade(R)

# The STEPS nowcast
nowcast_method = nowcasts.get_method("steps")
R_f_ens_v2 = nowcast_method(
    R[-3:, :, :],
    V,
    n_leadtimes,
    n_ens_members,
    n_cascade_levels=6,
    precip_thr=precip_thr,
    kmperpixel=2,
    timestep=timestep,
    # noise_method="nonparametric",
    vel_pert_method="bps",
    mask_method="incremental",
    seed=seed,
    extrap_method='semilagrangian',
    num_workers=2,
    probmatching_method="cdf"
)

# Back-transform to rain rates
# R_f_ens_v2 = transformation.dB_transform(R_f_ens_v2, threshold=-10.0, inverse=True)[0]

end_time = time.time()
execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")
method = 'STEPS Ens v2'
df = pd.read_csv(r"G:\My Drive\PowPow\Resort Graphs\forecast_radar\calc_duration.csv")
# Create a new row with the variables as columns
new_row = {"method": method,"num_prev_files": num_prev_files,"n_ens_members": n_ens_members,"n_leadtimes": n_leadtimes,"seed": seed,"execution_time": execution_time,}

# Convert the new row to a DataFrame
new_row_df = pd.DataFrame([new_row])

# Concatenate the original DataFrame with the new row DataFrame
df = pd.concat([df, new_row_df], ignore_index=True)

# Save the modified DataFrame to a new CSV file
df.to_csv(r"G:\My Drive\PowPow\Resort Graphs\forecast_radar\calc_duration.csv", index=False)

## Plot STEPS

In [ ]:
for i in range(R_f_ens.shape[1]):
    f_step = i + 1
    
    # Add 8 minutes from the datetime object
    new_date = date + timedelta(minutes=(f_step)*8)
    # Format the new date as a string
    new_date_string = new_date.strftime("%Y%m%d%H%M")

    # Get the extent values from metadata
    x1 = metadata['x1']
    x2 = metadata['x2']
    y2 = metadata['y1']
    y1 = metadata['y2']

    # Create a grid of x and y coordinates
    x = np.linspace(x1, x2, R.shape[2])
    y = np.linspace(y1, y2, R.shape[1])
    X, Y = np.meshgrid(x, y)

    bounds = [0.08, 0.16, 0.25,  0.40, 0.63, 1, 1.6, 2.5, 4, 6.3, 10, 16, 25, 40, 63, 100, 160, 220]
    norm = colors.BoundaryNorm(boundaries=bounds, ncolors=len(bounds))
    colormap = colors.ListedColormap(['#9e7c94', #grey
                                      '#650064','#ae00b0','#dc01da', #purples 5-15
                                      '#3531c3','#0b60f9', #blues
                                      '#009697', '#00c930','#65ff01','#98ff00','#c6fe00', #greens
                                      '#fffd04', #yellow
                                      '#ffc802', '#ffa000','#ff7d01', #orange
                                      '#df1a01','#8d0000' ]) #red           #65-75
    
    #Calulate the ensemble mean
    R_f_mean = np.mean(R_f_ens[:, i, :, :], axis=0)
    R_f_mean_v2 = np.mean(R_f_ens_v2[:, i, :, :], axis=0)
    
    # Create the figure and subplots
    # fig, axs = plt.subplots(1, 2, figsize=(18, 9), dpi=200, subplot_kw={'projection': ccrs.PlateCarree()})
    fig, axs = plt.subplots(2, 1, figsize=(12, 12), dpi=200, subplot_kw={'projection': ccrs.PlateCarree()})

    # Add features to both subplots
    for ax in axs:
        ax.add_feature(USCOUNTIES.with_scale('5m'), linewidth=.5, edgecolor='darkgrey')
        ax.add_feature(cartopy.feature.STATES, linewidth=1, edgecolor='black')

        # Set up the first subplot
        axs[0].set_title(new_date_string+" STEPS Ens (+ %i min)" % (f_step * timestep))
        axs[0].set_xlim(domain_lon)
        axs[0].set_ylim(domain_lat)

        # Set up the second subplot
        axs[1].set_title(new_date_string+" STEPS Ens v2 (+ %i min)" % (f_step * timestep))
        axs[1].set_xlim(domain_lon)
        axs[1].set_ylim(domain_lat)

        # Plot in the first subplot
        axs[0].contourf(X, Y, R_f_mean, levels=bounds, norm=norm, cmap=colormap)
        axs[1].contourf(X, Y, R_f_mean_v2, levels=bounds, norm=norm, cmap=colormap)
        
        # Plot 2.5mm contour
        axs[0].contour(X, Y, R_f_mean, levels=[.254], colors='black')
        axs[1].contour(X, Y, R_f_mean_v2, levels=[.254], colors='black')
    
    file_name="STEPS + STEPS v2  (+ %i min)" % (f_step * timestep)
    mrms_radar_path = os.path.join(r'G:\My Drive\PowPow\Resort Graphs\forecast_radar\ens', file_name)
    plt.savefig(mrms_radar_path+'.png',dpi=200,bbox_inches='tight')
    plt.close()

## STEPS Probabilities

In [ ]:
for i in range(R_f_ens.shape[1]):
    f_step = i + 1
    
    # Add 8 minutes from the datetime object
    new_date = date + timedelta(minutes=(f_step)*8)
    # Format the new date as a string
    new_date_string = new_date.strftime("%Y%m%d%H%M")

    # Get the extent values from metadata
    x1 = metadata['x1']
    x2 = metadata['x2']
    y2 = metadata['y1']
    y1 = metadata['y2']

    # Create a grid of x and y coordinates
    x = np.linspace(x1, x2, R.shape[2])
    y = np.linspace(y1, y2, R.shape[1])
    X, Y = np.meshgrid(x, y)
    
    bounds = [0, 0.1, 0.2,  0.3, 0.4, .5, .6, .7, .8, .9, 1]
    norm = colors.BoundaryNorm(boundaries=bounds, ncolors=len(bounds))
    colormap = colors.ListedColormap(['#FFFFFF','#fff5f0', '#fee0d2', '#fcbba1', '#fc9272', '#fb6a4a', '#ef3b2c', '#cb181d', '#a50f15', '#67000d'])

    P = excprob(R_f_ens[:, i, :, :], .254)
    P_v2 = excprob(R_f_ens_v2[:, i, :, :], .254)
    
    #Calulate the ensemble mean
    R_f_mean = np.mean(R_f_ens[:, i, :, :], axis=0)
    R_f_mean_v2 = np.mean(R_f_ens_v2[:, i, :, :], axis=0)
    
    # Create the figure and subplots
    # fig, axs = plt.subplots(1, 2, figsize=(18, 14), dpi=200, subplot_kw={'projection': ccrs.PlateCarree()})
    fig, axs = plt.subplots(2, 1, figsize=(12, 12), dpi=200, subplot_kw={'projection': ccrs.PlateCarree()})
    for ax in axs:
        ax.add_feature(USCOUNTIES.with_scale('5m'), linewidth=.5, edgecolor='darkgrey')
        ax.add_feature(cartopy.feature.STATES, linewidth=1, edgecolor='black')

        # Set up the first subplot
        axs[0].set_title(new_date_string+" Exceedence Probability .01 inch (+ %i min)" % (f_step * timestep))
        axs[0].set_xlim(domain_lon)
        axs[0].set_ylim(domain_lat)

        # Set up the second subplot
        axs[1].set_title(new_date_string+" v2 Exceedence Probability .01 inch (+ %i min)" % (f_step * timestep))
        axs[1].set_xlim(domain_lon)
        axs[1].set_ylim(domain_lat)

        # Plot in the first subplot
        axs[0].contourf(X, Y, P, levels=bounds, norm=norm, cmap=colormap)
        axs[1].contourf(X, Y, P_v2, levels=bounds, norm=norm, cmap=colormap)
        
        # Plot 2.5mm contour
        axs[0].contour(X, Y, R_f_mean, levels=[.254], colors='black')
        axs[1].contour(X, Y, R_f_mean_v2, levels=[.254], colors='black')

    # plt.tight_layout()
    file_name="Exceedence probability (+ %i min)" % (f_step * timestep)
    mrms_radar_path = os.path.join(r'G:\My Drive\PowPow\Resort Graphs\forecast_radar\prob', file_name)
    plt.savefig(mrms_radar_path+'.png',dpi=200,bbox_inches='tight')
    plt.close()

## Create Loops

In [ ]:
# Source folder directories
folder1 = r"G:\My Drive\PowPow\Resort Graphs\forecast_radar\mrms"
folder2 = r"G:\My Drive\PowPow\Resort Graphs\forecast_radar\deterministic" 

# Output GIF file path
output_gif = r"G:\My Drive\PowPow\Resort Graphs\forecast_radar\deterministic\case_studies\deterministic_"+domain_name+gif_date_start+".gif"
print(output_gif)
# List to store image paths
image_paths = []

# Function to get all PNG files from a folder
def get_png_files(folder):
    png_files = []
    for file in os.listdir(folder):
        if file.endswith(".png"):
            file_path = os.path.join(folder, file)
            png_files.append(file_path)
    return png_files

# Get PNG files from folder1
image_paths1 = get_png_files(folder1)

# Get PNG files from folder2
image_paths2 = get_png_files(folder2)

# Concatenate the image paths
image_paths = image_paths1 + image_paths2

# Sort the image paths in alphanumeric order
# image_paths.sort()

# Create GIF from the image paths
images = []
for image_path in image_paths:
    image = Image.open(image_path)
    images.append(image)

# Save the GIF with the original image resolution
images[0].save(output_gif, save_all=True, append_images=images[1:], optimize=False, duration=300, loop=0)

In [ ]:
# Source folder directories
folder1 = r"G:\My Drive\PowPow\Resort Graphs\forecast_radar\mrms"
folder2 = r"G:\My Drive\PowPow\Resort Graphs\forecast_radar\ens" 

# Output GIF file path
output_gif = r"G:\My Drive\PowPow\Resort Graphs\forecast_radar\ens\case_studies\ens_"+domain_name+gif_date_start+".gif"

# List to store image paths
image_paths = []

# Function to get all PNG files from a folder
def get_png_files(folder):
    png_files = []
    for file in os.listdir(folder):
        if file.endswith(".png"):
            file_path = os.path.join(folder, file)
            png_files.append(file_path)
    return png_files

# Get PNG files from folder1
image_paths1 = get_png_files(folder1)

# Get PNG files from folder2
image_paths2 = get_png_files(folder2)

# Concatenate the image paths
image_paths = image_paths1 + image_paths2

# Sort the image paths in alphanumeric order
# image_paths.sort()

# Create GIF from the image paths
images = []
for image_path in image_paths:
    image = Image.open(image_path)
    images.append(image)

# Save the GIF with the original image resolution
images[0].save(output_gif, save_all=True, append_images=images[1:], optimize=False, duration=300, loop=0)

In [ ]:
# Source folder directories
folder1 = r"G:\My Drive\PowPow\Resort Graphs\forecast_radar\mrms"
folder2 = r"G:\My Drive\PowPow\Resort Graphs\forecast_radar\prob" 

# Output GIF file path
output_gif = r"G:\My Drive\PowPow\Resort Graphs\forecast_radar\prob\case_studies\prob_"+domain_name+gif_date_start+".gif"

# List to store image paths
image_paths = []

# Function to get all PNG files from a folder
def get_png_files(folder):
    png_files = []
    for file in os.listdir(folder):
        if file.endswith(".png"):
            file_path = os.path.join(folder, file)
            png_files.append(file_path)
    return png_files

# Get PNG files from folder1
image_paths1 = get_png_files(folder1)

# Get PNG files from folder2
image_paths2 = get_png_files(folder2)

# Concatenate the image paths
image_paths = image_paths1 + image_paths2

# Sort the image paths in alphanumeric order
# image_paths.sort()

# Create GIF from the image paths
images = []
for image_path in image_paths:
    image = Image.open(image_path)
    images.append(image)

# Save the GIF with the original image resolution
images[0].save(output_gif, save_all=True, append_images=images[1:], optimize=False, duration=300, loop=0)

## Ens Members

In [ ]:
# Plot some of the realizations
fig = plt.figure()#figsize=(12, 12))
for i in range(20):
    ax = fig.add_subplot(5,4,i+1)#441 + i)
    ax = plot_precip_field(
        R_f_ens[i, -1, :, :], geodata=metadata, colorbar=False, axis="off",bbox = [-110, 36.7, -101, 41.5],
    )
    ax.set_title("Member %02d" % i)
plt.tight_layout()
plt.savefig('ens_co.png',dpi=200,bbox_inches='tight')
plt.show()

## LINDA Deterministic

In [ ]:
# import time
# start_time = time.time()

# from pysteps.nowcasts import linda, sprog, steps
# # Estimate the motion field
# from pysteps import motion
# V = dense_lucaskanade(R)
# # Compute 30-minute LINDA nowcast with 8 parallel workers
# # Restrict the number of features to 15 to reduce computation time
# nowcast_linda = linda.forecast(
#     R[-3:, :, :], #rainrate,
#     V, #advection,
#     n_leadtimes, #6,
#     max_num_features= 5, #15,
#     add_perturbations=False,
#     num_workers=12, #8
#     measure_time=True,
#     kmperpixel=4,
# )[0]

# end_time = time.time()
# execution_time = end_time - start_time
# method = 'LINDA Deterministic'
# df = pd.read_csv(r"G:\My Drive\PowPow\Resort Graphs\forecast_radar\calc_duration.csv")
# # Create a new row with the variables as columns
# new_row = {"method": method,"num_prev_files": num_prev_files,"n_ens_members": n_ens_members,"n_leadtimes": n_leadtimes,"seed": seed,"execution_time": execution_time,}
# # Convert the new row to a DataFrame
# new_row_df = pd.DataFrame([new_row])
# # Concatenate the original DataFrame with the new row DataFrame
# df = pd.concat([df, new_row_df], ignore_index=True)
# # Save the modified DataFrame to a new CSV file
# df.to_csv(r"G:\My Drive\PowPow\Resort Graphs\forecast_radar\calc_duration.csv", index=False)

## STEPS Ensemble Single Pane

In [ ]:
# # The STEPS nowcast
# nowcast_method = nowcasts.get_method("steps")
# R_f = nowcast_method(
#     R[-3:, :, :],
#     V,
#     n_leadtimes,
#     n_ens_members,
#     n_cascade_levels=6,
#     # R_thr=-10.0,
#     precip_thr=-10.0,
#     kmperpixel=2,
#     timestep=timestep,
#     noise_method="nonparametric",
#     vel_pert_method="bps",
#     mask_method="incremental",
#     seed=seed,
# )

# # Back-transform to rain rates
# R_f = transformation.dB_transform(R_f, threshold=-10.0, inverse=True)[0]

# for i in range(R_f.shape[1]):
#     f_step = i + 1
#     #fig size
#     plt.figure(figsize=(12, 6.5), dpi=100)
#     ax = plt.axes(projection=ccrs.PlateCarree())
#     ax.add_feature(USCOUNTIES.with_scale('5m'),linewidth=.5,edgecolor='darkgrey')
#     ax.add_feature(cartopy.feature.STATES,linewidth=1,edgecolor='black')
    
#     # Add 8 minutes from the datetime object
#     new_date = date + timedelta(minutes=(f_step)*8)
#     # Format the new date as a string
#     new_date_string = new_date.strftime("%Y%m%d%H%M")

#     # Get the extent values from metadata
#     x1 = metadata['x1']
#     x2 = metadata['x2']
#     y2 = metadata['y1']
#     y1 = metadata['y2']

#     # Create a grid of x and y coordinates
#     x = np.linspace(x1, x2, R.shape[2])
#     y = np.linspace(y1, y2, R.shape[1])
#     X, Y = np.meshgrid(x, y)

#     bounds = [0.08, 0.16, 0.25,  0.40, 0.63, 1, 1.6, 2.5, 4, 6.3, 10, 16, 25, 40, 63, 100, 160, 220]
#     norm = colors.BoundaryNorm(boundaries=bounds, ncolors=len(bounds))
#     colormap = colors.ListedColormap(['#9e7c94', #grey
#                                       '#650064','#ae00b0','#dc01da', #purples 5-15
#                                       '#3531c3','#0b60f9', #blues
#                                       '#009697', '#00c930','#65ff01','#98ff00','#c6fe00', #greens
#                                       '#fffd04', #yellow
#                                       '#ffc802', '#ffa000','#ff7d01', #orange
#                                       '#df1a01','#8d0000' ]) #red           #65-75
#     # print(R_f[:,:,:,:].shape)
#     # Plot the S-PROG forecast using contourf
#     R_f_mean = np.mean(R_f[:, i, :, :], axis=0)
#     # print(R_f_mean.shape)
#     plt.contourf(X, Y, R_f_mean, levels=bounds, norm=norm, cmap=colormap)
    
#     plt.title(new_date_string+" Ensemble mean (+ %i min)" % (f_step * timestep))
#     plt.colorbar(label='Rain Rate (mm/h)')

#     # Set the plot limits
#     plt.xlim(-110, -101)
#     plt.ylim(36.7, 41.5)
    
#     # # Plot the S-PROG forecast
#     # R_f_mean = np.mean(R_f[:, i, :, :], axis=0)
#     # plot_precip_field(
#     #     R_f_mean,
#     #     geodata=metadata,
#     #     title=new_date_string+"Ensemble mean (+ %i min)" % (f_step * timestep),
#     #     bbox = [-110, 36.7, -101, 41.5],
#     # )
#     file_name="Ensemble mean (+ %i min)" % (f_step * timestep)
#     mrms_radar_path = os.path.join(r'G:\My Drive\PowPow\Resort Graphs\forecast_radar\ens', file_name)
#     plt.savefig(mrms_radar_path+'.png',dpi=200,bbox_inches='tight')

In [ ]:
# # P = excprob(R_f[:, -1, :, :], 0.5)

# for i in range(R_f.shape[1]):
#     f_step = i + 1
#     #fig size
#     plt.figure(figsize=(12, 6.5), dpi=100)
#     ax = plt.axes(projection=ccrs.PlateCarree())
#     ax.add_feature(cartopy.feature.STATES,linewidth=1,edgecolor='darkgray')
#     ax.add_feature(USCOUNTIES.with_scale('5m'),linewidth=.5,edgecolor='darkgrey')
#     P = excprob(R_f[:, i, :, :], 2.54)
#     print(P.shape)
#     # Add 8 minutes from the datetime object
#     new_date = date + timedelta(minutes=(f_step)*8)
#     # Format the new date as a string
#     new_date_string = new_date.strftime("%Y%m%d%H%M")   
    
#     plot_precip_field(
#         P,
#         geodata=metadata,
#         ptype="prob",
#         units="mm/h",
#         probthr=2.54,
#         title=new_date_string+"Exceedence probability .1 inch (+ %i min)" % (f_step * timestep),
#         bbox = [-110, 36.7, -101, 41.5],
#     )
#     file_name="Exceedence probability (+ %i min)" % (f_step * timestep)
#     # mrms_radar_path = os.path.join(r'G:\My Drive\PowPow\Resort Graphs\forecast_radar\prob', file_name)
#     # plt.savefig(mrms_radar_path+'.png',dpi=200,bbox_inches='tight')

## MRMS Single Pane

In [ ]:
# date = datetime.strptime(formatted_datetime, "%Y%m%d%H%M")
# data_source = "mrms_os"      
# from pysteps.visualization.basemaps import plot_map_cartopy

# # Load data source config
# # data_source = "mrms_os"
# root_path = rcparams.data_sources[data_source]["root_path"]
# path_fmt = rcparams.data_sources[data_source]["path_fmt"]
# fn_pattern = rcparams.data_sources[data_source]["fn_pattern"]
# fn_ext = rcparams.data_sources[data_source]["fn_ext"]
# importer_name = rcparams.data_sources[data_source]["importer"]
# importer_kwargs = rcparams.data_sources[data_source]["importer_kwargs"]
# timestep = rcparams.data_sources[data_source]["timestep"]
# num_prev_files = 4

# # Find the radar files in the archive
# fns = io.find_by_date(
#     date, root_path, path_fmt, fn_pattern, fn_ext, timestep, num_prev_files
# )

# # Read the data from the archive
# importer = io.get_method(importer_name, "importer")
# R, _, metadata = io.read_timeseries(fns, importer, **importer_kwargs)

# # Convert to rain rate
# R, metadata = conversion.to_rainrate(R, metadata)

# # Upscale data to 2 km to limit memory usage
# # R, metadata = dimension.aggregate_fields_space(R, metadata, 2000)

# for i in range(R.shape[0]):
#     if i == 0:
#         f_step = 32
#     elif i == 1:
#         f_step = 24
#     elif i == 2:
#         f_step = 16
#     elif i == 3:
#         f_step = 8
#     elif i == 4:
#         f_step = 0

#     # fig size
#     plt.figure(figsize=(12, 6.5), dpi=200)
#     ax = plt.axes(projection=ccrs.PlateCarree())
#     ax.add_feature(USCOUNTIES.with_scale('5m'), linewidth=.5, edgecolor='darkgrey')
#     ax.add_feature(cartopy.feature.STATES, linewidth=1, edgecolor='black')

#     # Subtract 8 minutes from the datetime object
#     new_date = date - timedelta(minutes=(num_prev_files - i) * 8)
#     # Format the new date as a string
#     new_date_string = new_date.strftime("%Y%m%d%H%M")

#     # Get the extent values from metadata
#     x1 = metadata['x1']
#     x2 = metadata['x2']
#     y2 = metadata['y1']
#     y1 = metadata['y2']

#     # Create a grid of x and y coordinates
#     x = np.linspace(x1, x2, R.shape[2])
#     y = np.linspace(y1, y2, R.shape[1])
#     X, Y = np.meshgrid(x, y)

#     bounds = [0.08, 0.16, 0.25,  0.40, 0.63, 1, 1.6, 2.5, 4, 6.3, 10, 16, 25, 40, 63, 100, 160, 220]
#     norm = colors.BoundaryNorm(boundaries=bounds, ncolors=len(bounds))
#     colormap = colors.ListedColormap(['#9e7c94', #grey
#                                       '#650064','#ae00b0','#dc01da', #purples 5-15
#                                       '#3531c3','#0b60f9', #blues
#                                       '#009697', '#00c930','#65ff01','#98ff00','#c6fe00', #greens
#                                       '#fffd04', #yellow
#                                       '#ffc802', '#ffa000','#ff7d01', #orange
#                                       '#df1a01','#8d0000' ]) #red           #65-75
#     # Plot the S-PROG forecast using contourf
#     plt.contourf(X, Y, R[i, :, :], levels=bounds, norm=norm, cmap=colormap)

#     plt.title(new_date_string + " MRMS (-%i min)" % (f_step))
#     plt.colorbar(label='Rain Rate (mm/h)')

#     # Set the plot limits
#     plt.xlim(-110, -101)
#     plt.ylim(36.7, 41.5)

#     file_name=str(i)+"_MRMS (-%i min)" % (f_step)
#     mrms_radar_path = os.path.join(r'G:\My Drive\PowPow\Resort Graphs\forecast_radar\mrms', file_name)
#     plt.savefig(mrms_radar_path+'.png', dpi=200, bbox_inches='tight')
#     # plt.show()

# # Log-transform the data to unit of dBR, set the threshold to 0.1 mm/h,
# # set the fill value to -15 dBR
# R, metadata = transformation.dB_transform(R, metadata, threshold=0.1, zerovalue=-15.0)

# # Set missing values with the fill value
# R[~np.isfinite(R)] = -15.0

## Single Plane S-PROG

In [ ]:
# ## Single Plane S-PROG
# # Estimate the motion field
# from pysteps import motion
# V = dense_lucaskanade(R)

# nowcast_method = nowcasts.get_method("sprog")
# R_f = nowcast_method(
#     R[-3:, :, :],
#     V,
#     n_leadtimes,
#     n_cascade_levels=6,
#     # R_thr=-10.0,
#     precip_thr=-10.0,

# )

# # Back-transform to rain rate
# R_f = transformation.dB_transform(R_f, threshold=-10.0, inverse=True)[0]

# for i in range(R_f.shape[0]):
#     f_step = i + 1
#     #fig size
#     plt.figure(figsize=(12, 6.5), dpi=100)
#     ax = plt.axes(projection=ccrs.PlateCarree())
#     ax.add_feature(USCOUNTIES.with_scale('5m'),linewidth=.5,edgecolor='darkgrey')
#     ax.add_feature(cartopy.feature.STATES,linewidth=1,edgecolor='black')
    
#     # Add 8 minutes from the datetime object
#     new_date = date + timedelta(minutes=(f_step)*8)
#     # Format the new date as a string
#     new_date_string = new_date.strftime("%Y%m%d%H%M")

#     # Get the extent values from metadata
#     x1 = metadata['x1']
#     x2 = metadata['x2']
#     y2 = metadata['y1']
#     y1 = metadata['y2']

#     # Create a grid of x and y coordinates
#     x = np.linspace(x1, x2, R.shape[2])
#     y = np.linspace(y1, y2, R.shape[1])
#     X, Y = np.meshgrid(x, y)

#     bounds = [0.08, 0.16, 0.25,  0.40, 0.63, 1, 1.6, 2.5, 4, 6.3, 10, 16, 25, 40, 63, 100, 160, 220]
#     norm = colors.BoundaryNorm(boundaries=bounds, ncolors=len(bounds))
#     colormap = colors.ListedColormap(['#9e7c94', #grey
#                                       '#650064','#ae00b0','#dc01da', #purples 5-15
#                                       '#3531c3','#0b60f9', #blues
#                                       '#009697', '#00c930','#65ff01','#98ff00','#c6fe00', #greens
#                                       '#fffd04', #yellow
#                                       '#ffc802', '#ffa000','#ff7d01', #orange
#                                       '#df1a01','#8d0000' ]) #red           #65-75
#     # Plot the S-PROG forecast using contourf
#     plt.contourf(X, Y, R_f[i, :, :], levels=bounds, norm=norm, cmap=colormap)

#     plt.title(new_date_string+" S-PROG (+ %i min)" % (f_step * timestep))
#     plt.colorbar(label='Rain Rate (mm/h)')

#     # Set the plot limits
#     plt.xlim(-110, -101)
#     plt.ylim(36.7, 41.5)
    
#     # # Plot the S-PROG forecast
#     # plot_precip_field(
#     #     R_f[i, :, :],
#     #     geodata=metadata,
#     #     title=new_date_string+" S-PROG (+ %i min)" % (f_step * timestep),
#     #     bbox = [-110, 36.7, -101, 41.5],
#     # )
#     file_name="S-PROG (+ %i min)" % (f_step * timestep)
#     mrms_radar_path = os.path.join(r'G:\My Drive\PowPow\Resort Graphs\forecast_radar\deterministic', file_name)
#     plt.savefig(mrms_radar_path+'.png',dpi=200,bbox_inches='tight')

In [ ]:
# # Plot some of the realizations
# fig = plt.figure(figsize=(19, 10), dpi=100)
# for i in range(4):
#     ax = fig.add_subplot(221 + i)
#     ax = plot_precip_field(
#         R_f[i, -1, :, :], geodata=metadata, colorbar=False, axis="off", bbox = [-110, 36.7, -101, 41.5],
#     )
#     ax.set_title("Member %02d" % i)
# plt.tight_layout()
# plt.show()

In [ ]:
# date = datetime.strptime(formatted_datetime, "%Y%m%d%H%M")
# data_source = "mrms_os"      
# from pysteps.visualization.basemaps import plot_map_cartopy

# # Load data source config
# root_path = rcparams.data_sources[data_source]["root_path"]
# path_fmt = rcparams.data_sources[data_source]["path_fmt"]
# fn_pattern = rcparams.data_sources[data_source]["fn_pattern"]
# fn_ext = rcparams.data_sources[data_source]["fn_ext"]
# importer_name = rcparams.data_sources[data_source]["importer"]
# importer_kwargs = rcparams.data_sources[data_source]["importer_kwargs"]
# timestep = rcparams.data_sources[data_source]["timestep"]
# num_prev_files=4
# # Find the radar files in the archive
# fns = io.find_by_date(
#     date, root_path, path_fmt, fn_pattern, fn_ext, timestep, num_prev_files
# )
# # Read the data from the archive
# importer = io.get_method(importer_name, "importer")
# R, _, metadata = io.read_timeseries(fns, importer, **importer_kwargs)

# # Convert to rain rate
# R, metadata = conversion.to_rainrate(R, metadata)

# # Upscale data to 2 km to limit memory usage
# # R, metadata = dimension.aggregate_fields_space(R, metadata, 2000)

# for i in range(R.shape[0]):
#     if i == 0:
#         f_step = 32
#     elif i == 1:
#         f_step = 24
#     elif i == 2:
#         f_step = 16
#     elif i == 3:
#         f_step = 8
#     elif i == 4:
#         f_step = 0
#     #fig size
#     plt.figure(figsize=(12, 6.5), dpi=200)
#     ax = plt.axes(projection=ccrs.PlateCarree())
#     ax.add_feature(USCOUNTIES.with_scale('5m'),linewidth=.5,edgecolor='darkgrey')
#     ax.add_feature(cartopy.feature.STATES,linewidth=1,edgecolor='black')
    
#     # Subtract 8 minutes from the datetime object
#     new_date = date - timedelta(minutes=(num_prev_files-i)*8)
#     # Format the new date as a string
#     new_date_string = new_date.strftime("%Y%m%d%H%M")
    
#     # Plot the S-PROG forecast
#     plot_precip_field(
#         R[i, :, :],
#         geodata=metadata,
#         title=new_date_string+" MRMS (-%i min)" % (f_step),
#         bbox = [-110, 36.7, -101, 41.5],
#     )    
#     file_name=str(i)+"_MRMS (-%i min)" % (f_step)
#     mrms_radar_path = os.path.join(r'G:\My Drive\PowPow\Resort Graphs\forecast_radar\mrms', file_name)
#     plt.savefig(mrms_radar_path+'.png',dpi=200,bbox_inches='tight')
#     # plt.show()

# # Log-transform the data to unit of dBR, set the threshold to 0.1 mm/h,
# # set the fill value to -15 dBR
# R, metadata = transformation.dB_transform(R, metadata, threshold=0.1, zerovalue=-15.0)

# # Set missing values with the fill value
# R[~np.isfinite(R)] = -15.0

# # Nicely print the metadata
# # pprint(metadata)

In [ ]:
# # Compute exceedence probabilities for a 0.5 mm/h threshold
# P = excprob(R_f[:, -1, :, :], 0.5)

# # Plot the field of probabilities
# plt.figure(figsize=(19, 10), dpi=100)
# plot_precip_field(
#     P,
#     geodata=metadata,
#     ptype="prob",
#     units="mm/h",
#     probthr=0.5,
#     title="Exceedence probability (+ %i min)" % (n_leadtimes * timestep),
# )
# plt.show()

# # sphinx_gallery_thumbnail_number = 5

In [ ]:
# date = datetime.strptime(formatted_datetime, "%Y%m%d%H%M")
# data_source = "mrms_os"      
# from pysteps.visualization.basemaps import plot_map_cartopy

# # Load data source config
# # data_source = "mrms_os"
# root_path = rcparams.data_sources[data_source]["root_path"]
# path_fmt = rcparams.data_sources[data_source]["path_fmt"]
# fn_pattern = rcparams.data_sources[data_source]["fn_pattern"]
# fn_ext = rcparams.data_sources[data_source]["fn_ext"]
# importer_name = rcparams.data_sources[data_source]["importer"]
# importer_kwargs = rcparams.data_sources[data_source]["importer_kwargs"]
# timestep = rcparams.data_sources[data_source]["timestep"]
# num_prev_files = 4

# # Find the radar files in the archive
# fns = io.find_by_date(
#     date, root_path, path_fmt, fn_pattern, fn_ext, timestep, num_prev_files
# )

# # Read the data from the archive
# importer = io.get_method(importer_name, "importer")
# R, _, metadata = io.read_timeseries(fns, importer, **importer_kwargs)

# # Convert to rain rate
# R, metadata = conversion.to_rainrate(R, metadata)

# # Upscale data to 2 km to limit memory usage
# # R, metadata = dimension.aggregate_fields_space(R, metadata, 2000)

# for i in range(R.shape[0]):
#     if i == 0:
#         f_step = 32
#     elif i == 1:
#         f_step = 24
#     elif i == 2:
#         f_step = 16
#     elif i == 3:
#         f_step = 8
#     elif i == 4:
#         f_step = 0

#     # Subtract 8 minutes from the datetime object
#     new_date = date - timedelta(minutes=(num_prev_files - i) * 8)
#     # Format the new date as a string
#     new_date_string = new_date.strftime("%Y%m%d%H%M")

#     # Create the figure and subplots
#     fig, axs = plt.subplots(1, 2, figsize=(18, 9), dpi=200, subplot_kw={'projection': ccrs.PlateCarree()})

#     # Add features to both subplots
#     for ax in axs:
#         ax.add_feature(USCOUNTIES.with_scale('5m'), linewidth=.5, edgecolor='darkgrey')
#         ax.add_feature(cartopy.feature.STATES, linewidth=1, edgecolor='black')

#     # Iterate over the subplots
#     for ax in axs:
#         # # Subtract 8 minutes from the datetime object
#         # new_date = date - timedelta(minutes=(num_prev_files - i) * 8)
#         # # Format the new date as a string
#         # new_date_string = new_date.strftime("%Y%m%d%H%M")

#         # Get the extent values from metadata
#         x1 = metadata['x1']
#         x2 = metadata['x2']
#         y2 = metadata['y1']
#         y1 = metadata['y2']

#         # Create a grid of x and y coordinates
#         x = np.linspace(x1, x2, R.shape[2])
#         y = np.linspace(y1, y2, R.shape[1])
#         X, Y = np.meshgrid(x, y)

#         bounds = [0.08, 0.16, 0.25, 0.40, 0.63, 1, 1.6, 2.5, 4, 6.3, 10, 16, 25, 40, 63, 100, 160, 220]
#         norm = colors.BoundaryNorm(boundaries=bounds, ncolors=len(bounds))
#         colormap = colors.ListedColormap(['#9e7c94', '#650064', '#ae00b0', '#dc01da', '#3531c3', '#0b60f9',
#                                           '#009697', '#00c930', '#65ff01', '#98ff00', '#c6fe00', '#fffd04',
#                                           '#ffc802', '#ffa000', '#ff7d01', '#df1a01', '#8d0000'])

#         # Plot the S-PROG forecast using contourf
#         ax.contourf(X, Y, R[i, :, :], levels=bounds, norm=norm, cmap=colormap)

#         ax.set_title(new_date_string + " MRMS (-%i min)" % (f_step))
#         # plt.colorbar(label='Rain Rate (mm/h)')

#         # ax.set_colorbar(label='Rain Rate (mm/h)')

#         ax.set_xlim(-110, -101)
#         ax.set_ylim(36.7, 41.5)

#     # Adjust the spacing between subplots
#     plt.subplots_adjust(wspace=0.1)
#     # plt.colorbar(label='Rain Rate (mm/h)')
#     # Add a colorbar to the figure
#     # cbar = fig.colorbar(axs[0].collections[0], ax=axs, label='Rain Rate (mm/h)')

#     # Save the figure
#     file_name = str(i) + "_MRMS (-%i min)" % (f_step)
#     mrms_radar_path = os.path.join(r'G:\My Drive\PowPow\Resort Graphs\forecast_radar\mrms', file_name)
#     plt.savefig(mrms_radar_path + '.png', dpi=200, bbox_inches='tight')

# # Log-transform the data to unit of dBR, set the threshold to 0.1 mm/h,
# # set the fill value to -15 dBR
# R, metadata = transformation.dB_transform(R, metadata, threshold=0.1, zerovalue=-15.0)

# # Set missing values with the fill value
# R[~np.isfinite(R)] = -15.0

In [ ]:
# from datetime import datetime
# import numpy as np
# import matplotlib.pyplot as plt
# import os
# import numpy as np
# import pygrib

# folder_directory = r'C:\Users\16126\pysteps_data\mrms_op'

# # Get a list of all files in the folder directory
# files = os.listdir(folder_directory)

# # Filter the list to include only the .grib2 files
# grib2_files = [file for file in files if file.endswith('.grib2')]

# # Sort the files if needed
# grib2_files.sort()

# # Load the data and metadata from the grib2 files into lists
# data = []
# metadata = []
# for file in grib2_files:
#     file_path = os.path.join(folder_directory, file)
#     grbs = pygrib.open(file_path)
    
#     for grb in grbs:
#         data.append(grb.values)
#         metadata.append(grb)

#     grbs.close()

# # Concatenate the data into a single array
# R = np.concatenate(data)

# # Print the shape of the R array
# print("Shape of R:", R.shape)

# # Print the metadata for each message
# for i, grb in enumerate(metadata):
#     print(f"Metadata for message {i+1}:")
#     print(grb)
#     print()


# # Read the data from the archive
# # importer = io.get_method(importer_name, "importer")
# # R, _, metadata = io.read_timeseries(fns, importer, **importer_kwargs)

# # Convert to rain rate
# R, metadata = conversion.to_rainrate(R, metadata)

# print(R,metadata)
# # Upscale data to 2 km to limit memory usage
# R, metadata = dimension.aggregate_fields_space(R, metadata, 2000)

# # Plot the rainfall field
# plot_precip_field(R[-1, :, :], geodata=metadata)
# plt.show()

# # Log-transform the data to unit of dBR, set the threshold to 0.1 mm/h,
# # set the fill value to -15 dBR
# R, metadata = transformation.dB_transform(R, metadata, threshold=0.1, zerovalue=-15.0)

# # Set missing values with the fill value
# R[~np.isfinite(R)] = -15.0

# # Nicely print the metadata
# pprint(metadata)